In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import unicodedata
import string
import pandas as pd

In [2]:
data = pd.read_csv('data/train2014.csv')
data.drop(['img_fname'], axis=1, inplace=True)
data.head()

,captions
0,['Two ducks floating on top of a lake with bro...
1,['A long bill bird standing on a beach next to...
2,['Two birds sitting on a red towel on a shower...
3,['many elephants walking with a person with a ...
4,['A black and white bird sitting on top of a r...


In [5]:
data.captions[0]

"['Two ducks floating on top of a lake with brown water.', 'Two ducks are swimming on the lake next to each other.', 'Two geese swimming on the water and looking toward the camera.', 'Two ducks facing the same direction, wading in a pond.', 'Two geese resting in a body of water']"

In [24]:
tmp = data.captions[1]
tmp.split("'")

['[',
 'A long bill bird standing on a beach next to the ocean.',
 ', ',
 'A small bird with a long, skinny beak walks on the beach.',
 ', ',
 'Long beaked brown bird walking on a beach.',
 ', ',
 'a close up of a bird on a beach near a body of water',
 ', ',
 'A bird with a pink beak is on the beach.',
 ']']

In [28]:
tmp_tmp = []

for caption in data.captions:
    tmp = caption
    for sent in tmp.split("'"):
        tmp_tmp.append(sent)

In [31]:
all_captions = [tmp for tmp in tmp_tmp if len(tmp)>6]

In [44]:
bow = list(set(' '.join(all_captions).split(' ')))

In [45]:
bow

['',
 'geek',
 'birds,',
 'now.',
 'tr-oft',
 'Trio',
 'looked',
 'process.',
 'ATVs.',
 'keys',
 'read',
 'square',
 'basking',
 'slip',
 'needs',
 'lane',
 '"Montage',
 'words',
 'moss',
 'lit',
 'bottles.',
 'set',
 'pipeline',
 'coral.',
 'citizen',
 'screeching',
 'bedsheets',
 'Shetland',
 'over,',
 'angle',
 'waterfront',
 'distance.",',
 'multiple',
 'flag.',
 'fixing',
 'skiis',
 'listless',
 'outcropping.',
 'netted',
 'toilet.",',
 'skull',
 'especially,',
 'residence',
 'tall',
 'baseball.',
 'ensnaring',
 'bull',
 'russian',
 'small',
 'leading',
 'guarding',
 'blooms',
 'rumbles',
 'buffalo',
 'condiment',
 'bitten',
 'zeal',
 'shorter',
 'normal',
 'low.',
 'prices.',
 'attacked',
 'tin',
 'headphones',
 'trial.',
 'beams',
 'signs\\n',
 'Palace"',
 'straining',
 'scrappy',
 'rams.',
 'cling',
 'neighborhood.',
 'Nicely',
 'labs',
 'employee',
 'fuzzy',
 'doorway.',
 'camera.",',
 'antiques',
 'bird',
 'knot',
 'ferry',
 'eave',
 'stable',
 'octopuses',
 'mouth."]',
 'SP

In [37]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [46]:
import random

# Random item from a list
def randomChoice(bow):
    return bow[random.randint(1, len(bow) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
#     category = randomChoice(all_categories)
    word = randomChoice(bow)
    return word

In [47]:
def inputTensor(sent):
    tensor = torch.zeros(sent, 1, len(bow))
    for li in range(sent):
        w = sent[li]
        tensor[li][0][bow.find(w)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(sent):
    word_indexes = [bow.find(sent[li]) for li in range(1, len(sent))]
    word_indexes.append(len(bow) - 1) # EOS
    return torch.LongTensor(word_indexes)

In [48]:
def randomTrainingExample():
    line = randomTrainingPair()
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return input_line_tensor, target_line_tensor

In [49]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

In [50]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [53]:
randomTrainingExample()

TypeError: torch.zeros received an invalid combination of arguments - got (str, int, int), but expected one of:
 * (int ... size)
      didn't match because some of the arguments have invalid types: ([31;1mstr[0m, [31;1mint[0m, [31;1mint[0m)
 * (torch.Size size)


In [51]:
rnn = RNN(len(bow), 128, len(bow))

n_iters = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0


TypeError: torch.zeros received an invalid combination of arguments - got (str, int, int), but expected one of:
 * (int ... size)
      didn't match because some of the arguments have invalid types: ([31;1mstr[0m, [31;1mint[0m, [31;1mint[0m)
 * (torch.Size size)


In [18]:
max_length = 20

# Sample from a category and starting letter
def sample(start_word='The'):
    with torch.no_grad():  # no need to track history in sampling
#         category_tensor = categoryTensor(category)
        input = inputTensor(start_word)
        hidden = rnn.initHidden()

        output_name = start_word

        for i in range(max_length):
            output, hidden = rnn(input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == len(bow) - 1:
                break
            else:
                letter = bow[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name


In [19]:
def samples(start_word='The'):
    for start_w in start_word:
        print(sample(start_w))


In [24]:
samples('English', 'N')


Nerton
